## GPTModify

In [21]:
# import torch
# import torch.nn as nn
# from transformers import GPT2Model, GPT2Config

# class CustomGPT2Model(GPT2Model):
#     def __init__(self, config):
#         super(CustomGPT2Model, self).__init__(config)
#         # Create your custom embedding layer
#         self.wpv = nn.Embedding(config.vocab_size, config.n_embd)
#         # Initialize the weights as needed
        
#     def forward(self, input_ids, pv_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
#         # Run the base GPT-2 forward pass
#         gpt2_output = super().forward(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask)
        
#         # Get the embeddings from the GPT-2 output
#         gpt2_embeddings = gpt2_output.last_hidden_state
        
#         # Apply your custom embedding
#         pv_embedded = self.wpv(pv_ids)
        
#         # Concatenate or combine your custom embeddings with the GPT-2 embeddings
#         combined_embeddings = gpt2_embeddings + pv_embedded#torch.cat([gpt2_embeddings, pv_embedded], dim=1)
        
#         return combined_embeddings

In [22]:
# config = AutoConfig.from_pretrained('gpt2') # Load the GPT-2 configuration
# model = CustomGPT2Model(config)  # Instantiate your custom model

In [23]:
# model

In [24]:
# # Provide input to both GPT-2 and your custom embedding
# input_ids = torch.tensor([[1, 2, 3]])  # Example input_ids for GPT-2
# pv_ids = torch.tensor([[4, 5, 6]])  # Example custom input_ids

# output = model(input_ids, custom_input_ids)
# output.shape #bs, seq_len, emb_dim

In [25]:
# from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
# import torch
# import torch.nn as nn

# class CustomGPT2Model(AutoModelForCausalLM):
#     def __init__(self, config, *model_args, **kwargs):
#         super(CustomGPT2Model).__init__(config, *model_args, **kwargs)
#         # Create a custom embedding layer
#         self.wpv = nn.Embedding(config.vocab_size, config.n_embd)

#     def forward(self, input_ids, pv_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None):
        
#         # Run the base GPT-2 forward pass
#         gpt2_output = super().forward(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask)
        
#         # Get the embeddings from the GPT-2 output
#         gpt2_embeddings = gpt2_output.last_hidden_state
        
#         # Apply your custom embedding
#         pv_embedded = self.wpv(pv_ids)
#         combined_embeddings = gpt2_embeddings + pv_embedded#torch.cat([gpt2_embeddings, pv_embedded], dim=1)
        
#         return combined_embeddings
        
# config = AutoConfig.from_pretrained("gpt2")  # Load the GPT-2 configuration
# custom_model = CustomGPT2Model.from_config(config)
# custom_model

In [26]:
# # Encode input texts
# input_text = ["This is an example sentence."]
# custom_input_text = ["Custom input text."]
# input_ids = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)["input_ids"]
# custom_input_ids = tokenizer(custom_input_text, return_tensors="pt", padding=True, truncation=True)["input_ids"]
# # Generate outputs using your custom model
# outputs = custom_model(input_ids, input_ids=custom_input_ids)

In [28]:
# # Set the device to GPU if available
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# import torch
# import torch.nn as nn
# # from transformers import GPT2Model, GPT2Config
# from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

# class PrivateGPT2(AutoModelForCausalLM):
#     def __init__(self, config):
#         super(PrivateGPT2, self).__init__()
#         self.embed_dim = config.hidden_size
#         self.pv_embed    = nn.Embedding(config.vocab_size, self.embed_dim) 
#         self.transformer = GPT2Model(config)
#         self.lm_head     = nn.Linear(self.embed_dim, config.vocab_size, bias=False)

#     def forward(self, input_ids, private_mask=None, attention_mask=None, labels = None, noise_stddev = 0.2, noise = 'gaussian', device = device):
#         # Get token embeddings from GPT-2
#         input_embeded = self.transformer.wte(input_ids)
#         pv_embeded    = self.pv_embed(input_ids)
        
#         if private_mask is not None:
#             # Identify positions with value 1
#             positions_with_noise = private_mask == 1 #bs, seq_len
#             if noise == 'gaussian':
#                 # Generate Gaussian noise
#                 noise_gauss = torch.normal(mean=0.0, std=noise_stddev, size=pv_embeded.size()).to(device) #bs, seq_len, emb_dim
#                 noise_pv = noise_gauss * positions_with_noise.unsqueeze(-1)  # Broadcasting noise to embeddings shape
#             else:
#                 ones = torch.ones(size=pv_embeded.size()).to(device) #bs, seq_len, emb_dim
#                 noise_pv = ones * positions_with_noise.unsqueeze(-1)  # Broadcasting noise to embeddings shape
#             # Add noise to the original embeddings
#             pv_embeded = pv_embeded * noise_pv #element wise multiplication #bs, seq_len, emb_dim

#         token_embeddings = input_embeded + pv_embeded
#         # Pass through the rest of the GPT-2 model
#         transformer_outputs = self.transformer(inputs_embeds=token_embeddings, attention_mask=attention_mask)
#         hidden_states = transformer_outputs.last_hidden_state
#         # print(hidden_states.shape)
#         lm_logits = self.lm_head(hidden_states)
        
#         loss = None
#         if labels is not None:
#             # move labels to correct device to enable model parallelism
#             labels = labels.to(lm_logits.device)
#             # Shift so that tokens < n predict n
#             shift_logits = lm_logits[..., :-1, :].contiguous()
#             shift_labels = labels[..., 1:].contiguous()
#             # Flatten the tokens
#             loss_fct = nn.CrossEntropyLoss()
#             loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
#         # return {'loss': loss, 'logits': lm_logits} 
#         return loss, lm_logits 
        
# # Example usage
# config = GPT2Config.from_pretrained('gpt2')
# model = PrivateGPT2.from_config(config)

In [29]:
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import torch
import torch.nn as nn

In [30]:
# from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
# import torch
# import torch.nn as nn
# import spacy

# class GPT2WithPrivacyEmbeddings:
#     def __init__(self, model_name):
#         self.tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#         self.model = GPT2LMHeadModel.from_pretrained(model_name)
#         self.nlp = spacy.load("en_core_web_sm")

#         # Add a privacy embedding layer
#         self.privacy_embedding = nn.Embedding(2, self.model.config.hidden_size)

#         # Define the types of named entities to consider as privacy-related
#         self.privacy_entity_types = set([
#             "PERSON", "ORG", "GPE", "FAC", "LOC"  # Add more as needed
#         ])

#     def generate_with_privacy(self, input_ids):
#         # Convert input_ids back to text
#         input_text = self.tokenizer.decode(input_ids, skip_special_tokens=True)
#         # Perform NER on the input text
#         doc = self.nlp(input_text)
#         assert len(doc) == len(input_ids)
        
#         # Create privacy mask based on named entity types
#         privacy_mask = [1 if token.ent_type_ in self.privacy_entity_types else 0 for token in doc]
#         privacy_mask = torch.tensor(privacy_mask)

#         privacy_embeds = self.privacy_embedding(privacy_mask)
#         combined_embeds = self.model.transformer.wte(input_ids)

#         # Apply privacy embeddings to corresponding positions
#         privacy_indices = torch.arange(len(input_ids))[privacy_mask == 1]
#         combined_embeds.index_add_(dim=1, index=privacy_indices.unsqueeze(0), source=privacy_embeds)

#         outputs = self.model(inputs_embeds=combined_embeds)
#         return outputs.last_hidden_state

# # Example usage
# gpt2_privacy = GPT2WithPrivacyEmbeddings('gpt2')

# # Generate text with privacy embeddings
# input_text = "John Doe's address is 123 Main Street."
# input_ids = gpt2_privacy.tokenizer.encode(input_text, add_special_tokens=True, return_tensors='pt').squeeze()

In [32]:
# privacy_entity_types = set([
#             "PERSON", "ORG", "GPE", "FAC", "LOC"  # Add more as needed
#         ])

In [33]:
# privacy_mask = [1 if token.ent_type_ in privacy_entity_types else 0 for token in doc]
# privacy_mask = torch.tensor(privacy_mask)
# privacy_mask

## BERT

In [22]:
from transformers import BertTokenizer, BertForMaskedLM

class Agrument:
    def __init__(self):
        self.bert_model_path = "bert-base-uncased"
        self.data_dir ="./data/SST-2/"
        self.sensitive_word_percentage = 1
        self.epsilon = 15
        
args = Agrument()

In [23]:
tokenizer  = BertTokenizer.from_pretrained(args.bert_model_path)
tokenizer_type = "subword"

model=BertForMaskedLM.from_pretrained(args.bert_model_path)
embedding_matrix = model.bert.embeddings.word_embeddings.weight.data.cpu().numpy()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# model

In [25]:
embedding_matrix.shape #vocab_size, emb_dim

(30522, 768)

In [26]:
from tqdm import tqdm
import os
import unicodedata
from collections import Counter

def get_vocab_SST2(data_dir,tokenizer,tokenizer_type="subword"):
    vocab=Counter()
    for split in ['train','dev']:
        data_file_path=os.path.join(data_dir,split+".tsv")
        num_lines = sum(1 for _ in open(data_file_path))
        with open(data_file_path, 'r') as csvfile:
                next(csvfile)
                for line in tqdm(csvfile,total=num_lines-1):
                    line=line.strip().split("\t")
                    text = line[0]
                    if tokenizer_type=="subword":
                        tokenized_text = tokenizer.tokenize(text)
                    elif tokenizer_type=="word":
                        tokenized_text = [token.text for token in tokenizer(text)]
                    for token in tokenized_text:
                        vocab[token]+=1
    if tokenizer_type == "subword":
        for token in tokenizer.vocab:
            vocab[token]+=1
    return vocab
    
vocab = get_vocab_SST2(args.data_dir, tokenizer, tokenizer_type=tokenizer_type)

100%|██████████| 872/872 [00:00<00:00, 7907.50it/s]


In [27]:
sensitive_word_count = int(args.sensitive_word_percentage * len(vocab))
words = [key for key, _ in vocab.most_common()]
sensitive_words = words[-sensitive_word_count - 1:]

sensitive_words2id = {word: k for k, word in enumerate(sensitive_words)}
print("#Total Words: %d, #Sensitive Words: %d" % (len(words),len(sensitive_words2id)))

#Total Words: 30522, #Sensitive Words: 30522


In [29]:
sensitive_word_embed = []
all_word_embed=[]

word2id = {}
sword2id = {}
sensitive_count = 0
all_count = 0
for cur_word in tokenizer.vocab:
    if cur_word in vocab and cur_word not in word2id:
        word2id[cur_word] = all_count
        emb = embedding_matrix[tokenizer.convert_tokens_to_ids(cur_word)]
        all_word_embed.append(emb)
        all_count += 1

        if cur_word in sensitive_words2id:
                sword2id[cur_word] = sensitive_count
                sensitive_count += 1
                sensitive_word_embed.append(emb)
        assert len(word2id) == len(all_word_embed)
        assert len(sword2id) == len(sensitive_word_embed)

In [30]:
import numpy as np
all_word_embed=np.array(all_word_embed, dtype='f')
sensitive_word_embed = np.array(sensitive_word_embed, dtype='f')

print("All Word Embedding Matrix: %s" % str(all_word_embed.shape))
print("Sensitive Word Embedding Matrix: %s" % str(sensitive_word_embed.shape))

from scipy.special import softmax
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
def cal_probability(word_embed_1, word_embed_2, epsilon=2.0):
    distance = euclidean_distances(word_embed_1, word_embed_2)
    sim_matrix = -distance
    prob_matrix = softmax(epsilon * sim_matrix / 2, axis=1)
    return prob_matrix

print("Calculating Prob Matrix for Exponential Mechanism...")
prob_matrix = cal_probability(all_word_embed, sensitive_word_embed, args.epsilon)

All Word Embedding Matrix: (30522, 768)
Sensitive Word Embedding Matrix: (30522, 768)
Calculating Prob Matrix for Exponential Mechanism...


In [31]:
prob_matrix.shape

(30522, 30522)

In [32]:
prob_matrix

array([[3.5674386e-03, 9.6986000e-04, 9.8571449e-04, ..., 3.8443824e-05,
        3.0685129e-05, 1.0674237e-06],
       [1.0004048e-03, 3.6797917e-03, 9.8736666e-04, ..., 3.9825387e-05,
        3.1109194e-05, 1.0706297e-06],
       [9.9128089e-04, 9.6262537e-04, 3.5875842e-03, ..., 3.8657694e-05,
        3.2715223e-05, 1.0582394e-06],
       ...,
       [6.7392748e-04, 6.7683042e-04, 6.7387125e-04, ..., 6.2537871e-02,
        3.8942115e-04, 6.5036838e-06],
       [6.1992632e-04, 6.0930417e-04, 6.5722846e-04, ..., 4.4879198e-04,
        7.2072342e-02, 5.4097072e-06],
       [1.5426155e-04, 1.5000072e-04, 1.5207547e-04, ..., 5.3615910e-05,
        3.8697464e-05, 5.1555777e-01]], dtype=float32)

## GPT2

In [1]:
from transformers import GPT2Model, GPT2Tokenizer

class Agrument:
    def __init__(self):
        self.gpt_model_path = "gpt2"
        self.data_dir ="./data/SST-2/"
        self.sensitive_word_percentage = 0.1
        self.epsilon = 15
        
args = Agrument()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer  = GPT2Tokenizer.from_pretrained(args.gpt_model_path)
tokenizer_type = "subword"

model = GPT2Model.from_pretrained(args.gpt_model_path)
# Access the model's embedding layer
embedding_layer = model.get_input_embeddings()
# Get the embedding matrix
embedding_matrix = embedding_layer.weight.data.cpu().numpy()

In [3]:
# embedding_layer_ref = embedding_layer
# embedding_layer_ref.weight

In [4]:
embedding_matrix.shape #vocab_size, emb

(50257, 768)

In [5]:
from tqdm import tqdm
import os
import unicodedata
from collections import Counter

def get_vocab_SST2(data_dir,tokenizer,tokenizer_type="subword"):
    vocab=Counter()
    for split in ['train','dev']:
        data_file_path=os.path.join(data_dir,split+".tsv")
        num_lines = sum(1 for _ in open(data_file_path))
        with open(data_file_path, 'r') as csvfile:
                next(csvfile)
                for line in tqdm(csvfile,total=num_lines-1):
                    line=line.strip().split("\t")
                    text = line[0]
                    if tokenizer_type=="subword":
                        tokenized_text = tokenizer.tokenize(text)
                    elif tokenizer_type=="word":
                        tokenized_text = [token.text for token in tokenizer(text)]
                    for token in tokenized_text:
                        vocab[token]+=1
    if tokenizer_type == "subword":
        for token in tokenizer.get_vocab():
            vocab[token]+=1
    return vocab
    
vocab = get_vocab_SST2(args.data_dir, tokenizer, tokenizer_type=tokenizer_type)

100%|██████████| 872/872 [00:00<00:00, 18358.40it/s]


In [6]:
vocab.most_common()[:10]

[('Ġ', 68282),
 ('Ġ,', 24998),
 ('Ġthe', 23450),
 ('Ġand', 20306),
 ('Ġa', 17420),
 ('Ġof', 17307),
 ('Ġ.', 13518),
 ('-', 12715),
 ('Ġto', 11970),
 ("Ġ'", 10388)]

In [7]:
sensitive_word_count = int(args.sensitive_word_percentage * len(vocab))
words = [key for key, _ in vocab.most_common()]
sensitive_words = words[-sensitive_word_count - 1:]

sensitive_words2id = {word: k for k, word in enumerate(sensitive_words)}
print("#Total Words: %d, #Sensitive Words: %d" % (len(words),len(sensitive_words2id)))

#Total Words: 50257, #Sensitive Words: 5026


In [8]:
sensitive_word_embed = []
all_word_embed=[]

word2id = {}
sword2id = {}
sensitive_count = 0
all_count = 0
for cur_word in tokenizer.get_vocab():
    if cur_word in vocab and cur_word not in word2id:
        word2id[cur_word] = all_count
        emb = embedding_matrix[tokenizer.convert_tokens_to_ids(cur_word)]
        all_word_embed.append(emb)
        all_count += 1

        if cur_word in sensitive_words2id:
                sword2id[cur_word] = sensitive_count
                sensitive_count += 1
                sensitive_word_embed.append(emb)
        assert len(word2id) == len(all_word_embed)
        assert len(sword2id) == len(sensitive_word_embed)

In [9]:
import numpy as np
all_word_embed=np.array(all_word_embed, dtype='f')
sensitive_word_embed = np.array(sensitive_word_embed, dtype='f')

print("All Word Embedding Matrix: %s" % str(all_word_embed.shape))
print("Sensitive Word Embedding Matrix: %s" % str(sensitive_word_embed.shape))

from scipy.special import softmax #softmax(x) = np.exp(x)/sum(np.exp(x))
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
def cal_probability(word_embed_1, word_embed_2, epsilon=2.0):
    distance = euclidean_distances(word_embed_1, word_embed_2)
    sim_matrix = -distance #inverse protional to the distance between x and y
    prob_matrix = softmax(epsilon * sim_matrix / 2, axis=1)
    return prob_matrix

print("Calculating Prob Matrix for Exponential Mechanism...")
prob_matrix = cal_probability(all_word_embed, sensitive_word_embed, args.epsilon)

All Word Embedding Matrix: (50257, 768)
Sensitive Word Embedding Matrix: (5026, 768)
Calculating Prob Matrix for Exponential Mechanism...


In [11]:
prob_matrix.shape

(50257, 5026)

*Note If we know which idx is a sensitive word already*
*Can we add noise during training when model found these idxs*

## Rerference
https://github.com/xiangyue9607/SanText/tree/main
https://github.com/xiangyue9607/SanText/blob/main/utils.py#L19
https://github.com/xiangyue9607/SanText/blob/main/SanText.py#L31

In [119]:
import torch
import torch.nn as nn

# Define the vocabulary size and embedding dimensions
vocab_size = 50257  # GPT-2's vocabulary size
embedding_dim = 768  # GPT-2's embedding dimensions

# Create an instance of the nn.Embedding module
embedding_matrix = nn.Embedding(vocab_size, embedding_dim)

In [122]:
embedding_matrix #vocab_size, emb_dim

Embedding(50257, 768)

In [124]:
# Example: Create embeddings for an input tensor with shape (2, 1024)
input_indices = torch.randint(0, vocab_size, (2, 1024))  # Replace with your desired input
print("Input indices shape:", input_indices.shape) #bs, seq_len

Input indices shape: torch.Size([2, 1024])


In [125]:
embeddings = embedding_matrix(input_indices)
print("Embeddings shape:", embeddings.shape) #bs, seq_len, emb_dim

Embeddings shape: torch.Size([2, 1024, 768])
